# Import required packages

In [1]:
#pip install opencv-python

In [2]:
# update-notebook-instance --recolourize --

In [3]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import gc

In [4]:
import tensorflow as tf

2023-08-31 11:17:32.798797: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-31 11:17:33.574564: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
from keras.preprocessing.image import ImageDataGenerator

# reading files from s3 bucket

In [6]:
training_data_uri = 's3://recolorising/small_set_LAB_3000.npy'

In [7]:
#Check the contents of the uri
# !aws s3 ls $training_data_uri/

In [8]:
from s3fs.core import S3FileSystem
s3 = S3FileSystem()

In [9]:
key = 'small_set_LAB_3000.npy'
bucket = 'recolorising'

full_img = np.load(s3.open('{}/{}'.format(bucket, key)))

In [10]:
#full_img.shape

converting lab into rgb for effective downsampling

In [11]:
full_img_rgb = []
for i in range(0, full_img.shape[0]):
    if i == 0:
        full_img_rgb = cv2.cvtColor(np.array(full_img[i,:,:,:]),cv2.COLOR_Lab2RGB)
        #print(full_img_rgb.shape)
        full_img_rgb = np.expand_dims(full_img_rgb,0)
    else:
        full_img_rgb = np.append(full_img_rgb,
                    np.expand_dims(cv2.cvtColor(np.array(full_img[i,:,:,:]),cv2.COLOR_Lab2RGB),0),axis=0)
full_img_rgb = tf.convert_to_tensor(full_img_rgb)

2023-08-31 11:21:59.815417: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Resizing the images to 128*128 p

In [12]:
#img = full_img_rgb[2, :, :, :]
#img = cv2.cvtColor(np.array(img), cv2.COLOR_Lab2RGB)
#plt.imshow(img)

In [13]:
full_img = tf.image.resize_with_pad(
                full_img_rgb,
                128,
                128,
                "bilinear"
                )

freeing up memory

In [14]:
del full_img_rgb
gc.collect()

43

img = full_img[2,:,:,:]
img = cv2.cvtColor(np.array(img), cv2.COLOR_Lab2RGB)
plt.imshow(np.array(img).astype('int'))

COnverting RGB back into Lab

In [15]:
full_img_lab = []
for i in range(0,full_img.shape[0]):
     if i==0:
          full_img_lab = cv2.cvtColor(np.array(full_img[i,:,:,:]),cv2.COLOR_RGB2Lab)
          #print(full_img_lab.shape)
          full_img_lab=np.expand_dims(full_img_lab,0)
     else:
          full_img_lab = np.append(full_img_lab,
                                         np.expand_dims(cv2.cvtColor(np.array(full_img[i,:,:,:]),cv2.COLOR_RGB2Lab),0),axis=0)
full_img = tf.convert_to_tensor(full_img_lab)

In [16]:
del full_img_lab
gc.collect()

0

# train and test split 70:30

In [17]:
full_img = full_img/255

In [18]:
train = full_img[:2200,:,:,:]
test = full_img[2200:,:,:,:]

Normalising the picture data

In [19]:
gray_train = train[:,:,:,1]
gray_test = test[:,:,:,1]

Write data to folder

In [20]:
np.save('Preprocessed/train.npy', train)
np.save('Preprocessed/test.npy', test)
np.save('Preprocessed/gray_train.npy', gray_train)
np.save('Preprocessed/gray_test.npy', gray_test)

Write data to s3 bucket

In [21]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = "recolorising"
prefix = "processed"

role = sagemaker.get_execution_role()

In [22]:
inputs = sagemaker_session.upload_data(path="Preprocessed", bucket=bucket, key_prefix=prefix)
print("input spec (in this case, just an S3 path): {}".format(inputs))

input spec (in this case, just an S3 path): s3://recolorising/processed
